In [1]:
import pandas as pd;
import scipy.stats as stats
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

/home/cabrera/anaconda3/envs/custev/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
a1_color = '#664c85'
a2_color = '#5b92b6'
a3_color = '#52d5bd'
a4_color = '#eeeeee'
a1_palette = 'mako'
a2_palette = 'mako_r'

## Market Basket Analysis

### Online retail

#### Initialization

In [3]:
df_onlineretail = pd.read_csv(r'../../../data/csv/onlineretail.csv', low_memory = False)
df_onlineretail

,invoiceno,stockcode,description,quantity,invoicedate,unitprice,customer,country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,1 12 2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,1 12 2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,1 12 2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,1 12 2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,1 12 2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,NaN,12,9 12 2011 12:50,0.85,12680.0,France
541905,581587,22899,NaN,6,9 12 2011 12:50,2.10,12680.0,France
541906,581587,23254,NaN,4,9 12 2011 12:50,4.15,12680.0,France
541907,581587,23255,NaN,4,9 12 2011 12:50,4.15,12680.0,France


In [4]:
# Getting numerical and categorical columns
numerical_columns_df_onlineretail = df_onlineretail.select_dtypes(include = np.number).columns.tolist()
categorical_columns_df_onlineretail = df_onlineretail[~df_onlineretail.isin(numerical_columns_df_onlineretail)].columns.tolist()

In [5]:
# Filtering where invoiceno has no 'C'
df_onlineretail = df_onlineretail[~df_onlineretail['invoiceno'].str.contains('C')]
df_onlineretail['invoiceno'] = df_onlineretail['invoiceno'].astype('str')

/tmp/ipykernel_11099/1441241386.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_onlineretail['invoiceno'] = df_onlineretail['invoiceno'].astype('str')


#### Analysis

In [6]:
basket_df_onlineretail = (
    df_onlineretail.groupby('country').get_group('France')
    .groupby(['invoiceno', 'description'])['quantity']
    .sum()
    .unstack()
    .reset_index()
    .fillna(0)
    .set_index('invoiceno')
)

basket_df_onlineretail

description,SET 2 TEA TOWELS I LOVE LONDON,ALARM CLOCK BAKELIKE GREEN,ALARM CLOCK BAKELIKE PINK,ALARM CLOCK BAKELIKE RED,CHARLOTTE BAG DOLLY GIRL DESIGN,CIRCUS PARADE LUNCH BOX,INFLATABLE POLITICAL GLOBE,LUNCH BOX I LOVE LONDON,MINI JIGSAW CIRCUS PARADE,MINI JIGSAW SPACEBOY,MINI PAINT SET VINTAGE,PANDA AND BUNNIES STICKER SHEET,POSTAGE,RED TOADSTOOL LED NIGHT LIGHT,ROUND SNACK BOXES SET OF4 WOODLAND,SET/2 RED RETROSPOT TEA TOWELS,SPACEBOY LUNCH BOX,STARS GIFT TAPE,VINTAGE HEADS AND TAILS CARD GAME,VINTAGE SEASIDE JIGSAW PUZZLES
invoiceno,,,,,,,,,,,,,,,,,,,,
536370,24,12,24,24,20,24,48,24,24,24,36,12,3,24,24,18,24,24,24,12


In [7]:
basketsets_df_onlineretail = basket_df_onlineretail.applymap(lambda x: 0 if x <= 0 else 1)
basketsets_df_onlineretail

description,SET 2 TEA TOWELS I LOVE LONDON,ALARM CLOCK BAKELIKE GREEN,ALARM CLOCK BAKELIKE PINK,ALARM CLOCK BAKELIKE RED,CHARLOTTE BAG DOLLY GIRL DESIGN,CIRCUS PARADE LUNCH BOX,INFLATABLE POLITICAL GLOBE,LUNCH BOX I LOVE LONDON,MINI JIGSAW CIRCUS PARADE,MINI JIGSAW SPACEBOY,MINI PAINT SET VINTAGE,PANDA AND BUNNIES STICKER SHEET,POSTAGE,RED TOADSTOOL LED NIGHT LIGHT,ROUND SNACK BOXES SET OF4 WOODLAND,SET/2 RED RETROSPOT TEA TOWELS,SPACEBOY LUNCH BOX,STARS GIFT TAPE,VINTAGE HEADS AND TAILS CARD GAME,VINTAGE SEASIDE JIGSAW PUZZLES
invoiceno,,,,,,,,,,,,,,,,,,,,
536370,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


##### Apriori algorithm

In [8]:
# My device dies in computing this
frequent_itemsets_df_onlineretail = apriori(basketsets_df_onlineretail, min_support = 0.07, use_colnames = True)
association_rules_df_onlineretail = association_rules(frequent_itemsets_df_onlineretail, metric = 'lift', min_threshold = 1)